In [202]:
import time
import rasterio
import numpy as np
import math
from pathlib import Path 
from osgeo import gdal
import glob
import geopandas as gpd
import pandas as pd
import sys

# 0. Data Check
## treepoint data list
- manup_treepoint.shp
- mandown_treepoint.shp
- br_rest1_treepoint.shp
- br_rest2_treepoint.shp

In [203]:
tp_up = gpd.GeoDataFrame.from_file("tree_points/manup_treepoint.shp")
tp_down = gpd.GeoDataFrame.from_file("tree_points/mandown_treepoint.shp")
tp_br1 = gpd.GeoDataFrame.from_file("tree_points/br_rest1_treepoint.shp")
tp_br2 = gpd.GeoDataFrame.from_file("tree_points/br_rest2_treepoint.shp")
tpall = gpd.GeoDataFrame( pd.concat( [tp_up, tp_down, tp_br1, tp_br2], ignore_index=True) )

In [204]:
# print(tp_up.info())
# print(tp_down.info())
# print(tp_br1.info())
# print(tp_br2.info())
# print(tpall.info())

## subclass change
- tpall (up, down, br) 모든 데이터 합쳐서 시작
- up[0:28273], down[28273:52401], br1[52401:74639], br2[74639:85799] (*종료점은 포함x)

In [205]:
#### 그냥 일단 합쳐서 만들기
# replace with *
tpall_sc = tpall['subclass'].fillna('*')

# subclass string to int
tpall_sc = tpall_sc.to_frame() # 열 하나만 추출하면 series로 type 바뀌어서 다시 dataframe으로 바꿔줘야 함.
sc_list = {'subclass':{'Rosidae':'1', 'Hamamelididae':'2', 'Asteridae':'3',
                           'Dilleniidae':'4', 'Magnoliidae':'5', '*':'6'}}
tpall_sc = tpall_sc.replace(sc_list) # sc_list의 내용을 (tp_up_sc)에 대체해라.
print(f'{tpall_sc.shape} {tpall_sc}')
    
# change .. lat이랑 long을 넣냐 안넣냐..
tpall.drop(["subclass", "GenusSpeci", "OBJECTID", "Lat", "Long"], axis = 1, inplace=True)
tpall = pd.concat([tpall, tpall_sc], axis = 1) #tpall_sc 먼저 쓰면 type이 dataframe으로 바뀌어서 포인트 샘플링을 못함. tpall 먼저 써야함.
print(tpall.info())

(85799, 1)       subclass
0            1
1            2
2            1
3            2
4            2
...        ...
85794        1
85795        6
85796        2
85797        1
85798        3

[85799 rows x 1 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85799 entries, 0 to 85798
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   geometry  85799 non-null  geometry
 1   subclass  85799 non-null  object  
dtypes: geometry(1), object(1)
memory usage: 1.3+ MB
None


In [206]:
type(tpall)

geopandas.geodataframe.GeoDataFrame

# 1. Point Sampling

In [207]:
# file에 영상 넣기 # NDVIup_file, NDVIdown_file, NDVIbr_file
for i in ['NDVI', 'SBI', 'GVI', 'YVI', 'WBI']:
    for j in ['up', 'down', 'br']:
        globals()[f'{i}{j}_file'] = '../KOMPSAT 위성영상/데이터 전처리/4. data/'+j+'/'+i+'.tif'
        globals()[f'{i}{j}_src'] = rasterio.open(globals()[f'{i}{j}_file'])

In [208]:
## 반복변수명과 값 확인
# for i in globals():
#     if i[-4:] == 'file':
#         print(f'{i}: {globals()[i]}')
#     elif i[-3:] == 'src':
#         print(f'{i}: {globals()[i]}')
#     else:
#         pass

In [209]:
# 각 파일의 데이터 개수 세기
datalen = [len(tp_up), len(tp_down), len(tp_br1), len(tp_br2)]
print(datalen)

[28273, 24128, 22238, 11160]


In [210]:
# 파일별로 tpall dataframe에 값 입력하기 (NDVIup, NDVIdown, NDVIbr)
# up[0:28273], down[28273:52401], br1[52401:74639], br2[74639:85799] (*종료점은 포함x)

## 동적할당...이 필요한 것 같아서ㅜㅜ 강의 듣고 다시 해보든가 해야할 듯.. 일단은 또 복붙..!

# for i in ['up', 'down', 'br']:
#     tpall[Path(globals()[f'NDVI{i}_file']).stem] = [x for x in globals()[f'NDVI{i}_src'](coord_list)] # 포인트 샘플링
#     tpall[Path(globals()[f'NDVI{i}_file']).stem] = tpall[Path(globals()[f'NDVI{i}_file']).stem].astype('float64')

In [211]:
tpup = tpall[:datalen[0]]
tpdown = tpall[datalen[0]:datalen[0]+datalen[1]]
tpbr = tpall[datalen[0]+datalen[1]:]
tpbr

,geometry,subclass
52401,POINT (-73.92916 40.61451),2
52402,POINT (-73.93893 40.61036),3
52403,POINT (-73.92881 40.62688),2
52404,POINT (-73.92100 40.61747),1
52405,POINT (-73.92101 40.61761),1
...,...,...
85794,POINT (-73.89033 40.65515),1
85795,POINT (-73.90517 40.64284),6
85796,POINT (-73.90518 40.64321),2
85797,POINT (-73.89036 40.65520),1


In [212]:
# just NDVI point sampling  # 포인트의 좌표를 리스트로 추출
coord_uplist = [(x,y) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)] 
coord_downlist = [(x,y) for x,y in zip(tpdown['geometry'].x , tpdown['geometry'].y)] 
coord_brlist = [(x,y) for x,y in zip(tpbr['geometry'].x , tpbr['geometry'].y)] 
print(f'{len(coord_uplist)}, {len(coord_downlist)}, {len(coord_brlist)}')

28273, 24128, 33398


In [218]:
tpup[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_uplist)] # 포인트 샘플링
tpup[Path(NDVIup_file).stem] = tpup[Path(NDVIup_file).stem].astype('float64')

tpdown[Path(NDVIdown_file).stem] = [x for x in NDVIdown_src.sample(coord_downlist)] # 포인트 샘플링
tpdown[Path(NDVIdown_file).stem] = tpdown[Path(NDVIdown_file).stem].astype('float64')

tpbr[Path(NDVIbr_file).stem] = [x for x in NDVIbr_src.sample(coord_brlist)] # 포인트 샘플링
tpbr[Path(NDVIbr_file).stem] = tpbr[Path(NDVIbr_file).stem].astype('float64')

# tpall = pd.concat([tpup, tpdown, tpbr])
# tpall

C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\admin\anaconda3\envs\sprout38\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,geometry,subclass,NDVI
0,POINT (-73.95018 40.80481),1,0.061114
1,POINT (-73.95173 40.80921),2,-0.068959
2,POINT (-73.94942 40.80449),1,-0.001804
3,POINT (-73.95062 40.80500),2,0.038803
4,POINT (-73.95038 40.80490),2,0.227972
...,...,...,...
85794,POINT (-73.89033 40.65515),1,0.092050
85795,POINT (-73.90517 40.64284),6,0.148645
85796,POINT (-73.90518 40.64321),2,0.002555
85797,POINT (-73.89036 40.65520),1,0.124759


In [219]:
# tpall에서 포인트 샘플링을 각각 하고? 0값이 나오면 

In [220]:
tpall

,geometry,subclass,NDVI
0,POINT (-73.95018 40.80481),1,0.061114
1,POINT (-73.95173 40.80921),2,-0.068959
2,POINT (-73.94942 40.80449),1,-0.001804
3,POINT (-73.95062 40.80500),2,0.038803
4,POINT (-73.95038 40.80490),2,0.227972
...,...,...,...
85794,POINT (-73.89033 40.65515),1,0.092050
85795,POINT (-73.90517 40.64284),6,0.148645
85796,POINT (-73.90518 40.64321),2,0.002555
85797,POINT (-73.89036 40.65520),1,0.124759


In [221]:
## tpall에서 NDVI에 0값 확인... 근데 다른 소숫점은 확인이 안되서 찝찝히지만 일단 패쓰
test = tpall[tpall['NDVI'] == 0]
test

,geometry,subclass,NDVI
4096,POINT (-73.94774 40.83100),2,0.0
5364,POINT (-73.94709 40.83103),4,0.0
5365,POINT (-73.94712 40.83100),2,0.0
7870,POINT (-73.94581 40.83100),1,0.0
7873,POINT (-73.94687 40.83102),1,0.0
...,...,...,...
50076,POINT (-73.96886 40.74797),2,0.0
50107,POINT (-73.96884 40.74818),2,0.0
50172,POINT (-73.96888 40.74825),2,0.0
52303,POINT (-73.99253 40.75932),1,0.0


## 이상치 제거: NDVI < 0.2 의 값을 가진 포인트 제거

In [222]:
## 모든 데이터 합쳐서 제거
# tp = tpall.drop(tpall[tpall['NDVI']<0.2].index)
# tp

,geometry,subclass,NDVI
4,POINT (-73.95038 40.80490),2,0.227972
8,POINT (-73.94979 40.80465),1,0.220261
9,POINT (-73.94973 40.80462),4,0.293061
10,POINT (-73.94993 40.80471),1,0.464282
13,POINT (-73.94916 40.80438),1,0.414272
...,...,...,...
85771,POINT (-73.90318 40.66614),1,0.367431
85788,POINT (-73.88648 40.65577),1,0.542233
85789,POINT (-73.88751 40.65815),2,0.232292
85790,POINT (-73.88742 40.65818),2,0.452200


In [233]:
tpup.drop(tpup[tpup['NDVI']<0.2].index, inplace=True)
tpdown.drop(tpdown[tpdown['NDVI']<0.2].index, inplace=True)
tpbr.drop(tpbr[tpbr['NDVI']<0.2].index, inplace=True)

C:\Users\admin\AppData\Local\Temp\ipykernel_16480\3660270846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tpup.drop(tpup[tpup['NDVI']<0.2].index, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16480\3660270846.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tpdown.drop(tpdown[tpdown['NDVI']<0.2].index, inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_16480\3660270846.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [234]:
tpdown

,geometry,subclass,NDVI
28274,POINT (-74.00383 40.73764),1,0.610439
28275,POINT (-73.97215 40.74586),1,0.423846
28281,POINT (-73.98022 40.74996),1,0.315406
28282,POINT (-73.97239 40.74907),6,0.253490
28283,POINT (-74.00006 40.74503),1,0.634895
...,...,...,...
52385,POINT (-74.00226 40.73641),4,0.710760
52386,POINT (-73.98430 40.71857),2,0.247676
52394,POINT (-74.00202 40.73149),1,0.413353
52396,POINT (-73.99764 40.72731),3,0.275474


## 3by3 커널의 픽셀값 가져오기
### 1. NDVI
### 2. SBI, GVI, YVI, WBI

In [223]:
# NDVI


In [236]:
WIDTH = 0.00000599299783809013749338974087784241
HEIGHT = 0.0000059299783809007439469768700121737
coord_list0= [(x,y) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list2= [(x, y+HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list4= [(x+WIDTH,y) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list6= [(x,y-HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]
coord_list8= [(x-WIDTH,y) for x,y in zip(tpup['geometry'].x , tpup['geometry'].y)]

In [237]:
for i in range(9):
    globals()[f'tp{i}'] = tp

In [247]:
def PointSampling(file, src, tp, feature):
    WIDTH = 0.00000599299783809013749338974087784241
    HEIGHT = 0.0000059299783809007439469768700121737
    coord_list0= [(x,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list2= [(x, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list4= [(x+WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list6= [(x,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    coord_list8= [(x-WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
    
    for i in range(9):
        globals()[f'tp{i}'] = tp
    
    for i in range(9):
        globals()[f'tp{i}'][Path(file).stem] = [x for x in src.sample(globals()[f'coord_list{i}'])] # 포인트 샘플링
        globals()[f'tp{i}'][Path(file).stem] = globals()[f'tp{i}'][Path(file).stem].astype('float64')
        globals()[f'tp{i}'].rename(columns={feature: feautre+'_'+i}, inplace=True)

In [248]:
PointSampling(NDVIdown_file, NDVIdown_src, tpdown, 'NDVI')

ValueError: Length of values (16979) does not match length of index (15042)

In [ ]:
tpdown

In [226]:
#1
tp0[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list0)] # 포인트 샘플링
tp0[Path(NDVIup_file).stem] = tp0[Path(NDVIup_file).stem].astype('float64')
tp0.rename(columns={'NDVI': 'NDVI_1'}, inplace=True)
tp0

,geometry,subclass,NDVI_1
4,POINT (-73.95038 40.80490),2,0.227972
8,POINT (-73.94979 40.80465),1,0.220261
9,POINT (-73.94973 40.80462),4,0.293061
10,POINT (-73.94993 40.80471),1,0.464282
13,POINT (-73.94916 40.80438),1,0.414272
...,...,...,...
85771,POINT (-73.90318 40.66614),1,0.000000
85788,POINT (-73.88648 40.65577),1,0.000000
85789,POINT (-73.88751 40.65815),2,0.000000
85790,POINT (-73.88742 40.65818),2,0.000000


In [198]:
# NDVI 0-8 numbering
#1
tp0[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list0)] # 포인트 샘플링
tp0[Path(NDVIup_file).stem] = tp0[Path(NDVIup_file).stem].astype('float64')
tp0.rename(columns={'NDVI': 'NDVI_1'}, inplace=True)
tp0.head()
print(len(tp0.index))

#2
tp1[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list1)] # 포인트 샘플링
tp1[Path(NDVIup_file).stem] = tp1[Path(NDVIup_file).stem].astype('float64')
tp1.rename(columns={'NDVI': 'NDVI_2'}, inplace=True)
tp1.head()
print(len(tp1.index))

#3
tp2[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list2)] # 포인트 샘플링
tp2[Path(NDVIup_file).stem] = tp2[Path(NDVIup_file).stem].astype('float64')
tp2.rename(columns={'NDVI': 'NDVI_3'}, inplace=True)
tp2.head()
print(len(tp2.index))

#4
tp3[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list3)] # 포인트 샘플링
tp3[Path(NDVIup_file).stem] = tp3[Path(NDVIup_file).stem].astype('float64')
tp3.rename(columns={'NDVI': 'NDVI_4'}, inplace=True)
tp3.head()
print(len(tp3.index))

#5
tp4[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list4)] # 포인트 샘플링
tp4[Path(NDVIup_file).stem] = tp4[Path(NDVIup_file).stem].astype('float64')
tp4.rename(columns={'NDVI': 'NDVI_5'}, inplace=True)
tp4.head()
print(len(tp4.index))

#6
tp5[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list5)] # 포인트 샘플링
tp5[Path(NDVIup_file).stem] = tp5[Path(NDVIup_file).stem].astype('float64')
tp5.rename(columns={'NDVI': 'NDVI_6'}, inplace=True)
tp5.head()
print(len(tp5.index))

#7
tp6[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list6)] # 포인트 샘플링
tp6[Path(NDVIup_file).stem] = tp6[Path(NDVIup_file).stem].astype('float64')
tp6.rename(columns={'NDVI': 'NDVI_7'}, inplace=True)
tp6.head()
print(len(tp6.index))

#8
tp7[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list7)] # 포인트 샘플링
tp7[Path(NDVIup_file).stem] = tp7[Path(NDVIup_file).stem].astype('float64')
tp7.rename(columns={'NDVI': 'NDVI_8'}, inplace=True)
tp7.head()

56049
56049
56049
56049
56049
56049
56049


,geometry,subclass,NDVI_1,NDVI_2,NDVI_3,NDVI_4,NDVI_5,NDVI_6,NDVI_7,NDVI_8
4,POINT (-73.95038 40.80490),2,0.227972,0.079281,0.137626,0.151640,0.252352,0.374262,0.343012,0.289679
8,POINT (-73.94979 40.80465),1,0.220261,0.175217,0.161357,0.117064,0.174200,0.246941,0.290238,0.308043
9,POINT (-73.94973 40.80462),4,0.293061,0.286925,0.204017,0.119694,0.143060,0.220945,0.289034,0.389648
10,POINT (-73.94993 40.80471),1,0.464282,0.409306,0.407115,0.452619,0.503392,0.578829,0.573110,0.534801
13,POINT (-73.94916 40.80438),1,0.414272,0.381289,0.316225,0.197326,0.245063,0.342866,0.342866,0.460873


In [ ]:
NDVI

In [46]:
for i in range(9):
    globals()[f'tp{i}'][Path(file).stem] = [x for x in src.sample(globals()[f'coord_list{i}'])] # 포인트 샘플링
    globals()[f'tp{i}'][Path(file).stem] = globals()[f'tp{i}'][Path(file).stem].astype('float64')
    globals()[f'tp{i}'].rename(columns={feature: feautre+'_'+i}, inplace=True)
        

In [ ]:
for i in range

In [61]:
# SBI, GVI, YVI, WBI
def pointsampling(area, file, src, feature):
    for i in range(0, 9):
        WIDTH = 0.00000599299783809013749338974087784241
        HEIGHT = 0.0000059299783809007439469768700121737
        coord_list0= [(x,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list1= [(x-WIDTH, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list2= [(x, y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list3= [(x+WIDTH,y+HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list4= [(x+WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list5= [(x+WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list6= [(x,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list7= [(x-WIDTH,y-HEIGHT) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        coord_list8= [(x-WIDTH,y) for x,y in zip(tp['geometry'].x , tp['geometry'].y)]
        
        # 합치는 것만 생각해바
        tp[Path(file).stem] = [x for x in src.sample(coord_list+'i')] # 포인트 샘플링
        tp[feature] = tp[feature].astype('float64') # 샘플링 값 dtype str to float64
        tp[feature].drop(tp[feature] == 0.0, inplace=True) 
        tp.rename(columns={feature: feautre+'_'+i}, inplace=True)
    tp.head()
    print(len(tp.index))
    return tp

In [63]:
tpup = pointsampling(tp_up, NDVIup_file, NDVIdown_src, coord_list, "NDVI")

AttributeError: 'float' object has no attribute 'index'

In [ ]:
ndvi_file = '../KOMPSAT 위성영상/데이터 전처리/4. data/manup2017/NDVI.tif'
ndvi_src = rasterio.open(ndvi_file) # 파일 읽기
tp[Path(ndvi_file).stem] = [x for x in ndvi_src.sample(coord_list)] # 포인트 샘플링
tp[Path(ndvi_file).stem] = tp[Path(ndvi_file).stem].astype('float64')

In [53]:
tp[Path(NDVIup_file).stem] = [x for x in NDVIup_src.sample(coord_list)] # 포인트 샘플링
#tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0          [0.061114295761074494]
1          [-0.06895863107852732]
2        [-0.0018042394869449067]
3          [0.038802622168380596]
4           [0.22797190493318442]
                   ...           
85794                       [0.0]
85795                       [0.0]
85796                       [0.0]
85797                       [0.0]
85798                       [0.0]
Name: NDVI, Length: 85799, dtype: object

In [54]:
tp[Path(NDVIup_file).stem] = tp[Path(NDVIup_file).stem].astype('float64')
tp[Path(NDVIup_file).stem]

0        0.061114
1       -0.068959
2       -0.001804
3        0.038803
4        0.227972
           ...   
85794    0.000000
85795    0.000000
85796    0.000000
85797    0.000000
85798    0.000000
Name: NDVI, Length: 85799, dtype: float64

In [55]:
type(tp[Path(NDVIup_file).stem])

pandas.core.series.Series